In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft
import numpy as np

In [ ]:
pxd_ = [
    np.asarray([[0], [1], [2]]),
    np.asarray([[0, -1], [1, 0], [2, 4]]),
    np.asarray([[0, -1, 8], [1, 0, 16], [2, 4, 24]]),
]

## get_probability

In [ ]:
for pxd in pxd_:

    kraft.probability.get_probability(pxd)

## get_posterior_probability

In [ ]:
for pxd in pxd_:

    kraft.probability.get_posterior_probability(pxd)

## plot_nomogram